In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../crawled_data/movies_data.csv')
df.head(2)

C:\Users\HIEPNT\AppData\Local\Temp\ipykernel_8784\4213205943.py:1: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../crawled_data/movies_data.csv')


,title,img_url,year,director,stars,duration,genres,overview,rating,num_rating,num_user_review,num_critic_review,budget,gross,country,metascore,oscar,win_and_nomination,url
0,The Godfather,https://m.media-amazon.com/images/M/MV5BM2MyNj...,1972,Francis Ford Coppola,"Marlon Brando,Al Pacino,James Caan,Marlon Bran...",2h 55m,"Crime,Drama","Don Vito Corleone, head of a mafia family, dec...",9.2,2M,5.3K,206.0,"$6,000,000 (estimated)","$250,341,816",United States,100.0,Won 3 Oscars,30 wins & 31 nominations total,https://www.imdb.com/title/tt0068646/?ref_=adv...
1,Interstellar,https://m.media-amazon.com/images/M/MV5BZjdkOT...,2014,Christopher Nolan,"Matthew McConaughey,Anne Hathaway,Jessica Chas...",2h 49m,"Adventure,Drama,Sci-Fi",When Earth becomes uninhabitable in the future...,8.7,2M,5.5K,486.0,"$165,000,000 (estimated)","$703,170,837",United States,74.0,Won 1 Oscar,44 wins & 148 nominations total,https://www.imdb.com/title/tt0816692/?ref_=adv...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222169 entries, 0 to 222168
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   title               222169 non-null  object 
 1   img_url             222169 non-null  object 
 2   year                222169 non-null  object 
 3   director            206284 non-null  object 
 4   stars               191677 non-null  object 
 5   duration            145469 non-null  object 
 6   genres              180838 non-null  object 
 7   overview            116945 non-null  object 
 8   rating              98497 non-null   float64
 9   num_rating          98497 non-null   object 
 10  num_user_review     53422 non-null   object 
 11  num_critic_review   20743 non-null   float64
 12  budget              39387 non-null   object 
 13  gross               13026 non-null   object 
 14  country             217128 non-null  object 
 15  metascore           2080 non-null 

In [4]:
# Ensure the 'year' column is of type str
df['year'] = df['year'].astype(str)

# Sort the DataFrame by 'year' in ascending order
df = df.sort_values(by='year')

In [5]:
# Đếm số lượng bộ phim có giá trị NULL (hoặc thiếu) trong mỗi trường
null_counts = df.isnull().sum()

# Hiển thị số lượng bộ phim có giá trị NULL trong từng trường
print("Null counts in each column:")
print(null_counts)

# Hiển thị số lượng bộ phim trước khi xóa bất kỳ
num_movies_before_deletion = len(df)
print(f"Number of movies before deletion: {num_movies_before_deletion}")

# Xóa bất kỳ bộ phim nào có giá trị NULL trong các trường quan trọng
important_columns = ['title', 'genres', 'rating', 'year', 'overview', 'duration']
df = df.dropna(subset=important_columns)

# Hiển thị số lượng bộ phim sau khi xóa
num_movies_after_deletion = len(df)
print(f"Number of movies after deletion: {num_movies_after_deletion}")

Null counts in each column:
title                      0
img_url                    0
year                       0
director               15885
stars                  30492
duration               76700
genres                 41331
overview              105224
rating                123672
num_rating            123672
num_user_review       168747
num_critic_review     201426
budget                182782
gross                 209143
country                 5041
metascore             220089
oscar                 191997
win_and_nomination    191996
url                        0
dtype: int64
Number of movies before deletion: 222169
Number of movies after deletion: 63737


In [6]:
len(df.columns)

19

In [7]:
# Define a function to convert 'K' and 'M' to integers
def convert_to_int(value):
    if 'K' in value:
        return int(float(value.replace('K', '')) * 1000)
    elif 'M' in value:
        return int(float(value.replace('M', '')) * 1000000)
    else:
        return int(value)

# Apply the function to convert 'num_rating' values to integers
df['num_rating'] = df['num_rating'].apply(convert_to_int)

In [8]:
# Lấy tất cả các giá trị duy nhất của cột 'year'
unique_years = df['year'].unique()

# xóa các giá trị không phải số trong cột 'year'
df = df[df['year'].str.isnumeric()]

# Chuyển cột 'year' thành kiểu dữ liệu số nguyên, bỏ .0
df['year'] = df['year'].astype(int)

In [9]:
# Define a function to convert 'duration' to minutes
def convert_duration_to_minutes(duration_str):
    parts = duration_str.split()
    total_minutes = 0
    for part in parts:
        if 'h' in part:
            total_minutes += int(part.strip('h')) * 60
        elif 'm' in part:
            total_minutes += int(part.strip('m'))
    return total_minutes

# Apply the function to convert 'duration' values to minutes
df['duration'] = df['duration'].apply(convert_duration_to_minutes)

In [10]:
# oscar count
# Define a function to extract the number of Oscars won
def extract_oscar_count(oscar_str):
    if isinstance(oscar_str, str) and 'oscar' in oscar_str.lower() and 'won' in oscar_str.lower():
        # Extract the number of Oscars won
        words = oscar_str.split()
        for i in range(len(words)):
            if words[i].isdigit() and words[i + 1].lower() == 'oscar' and words[i - 1].lower() == 'won':
                return int(words[i])
    # If not 'won' and 'oscar' or no number found, return 0
    return 0

# Apply the function to convert 'oscar' values to the number of Oscars won
df['oscar'] = df['oscar'].apply(extract_oscar_count)

In [11]:
#budget processing
df["budget"] = df["budget"].astype(str)
df["budget"] = df["budget"].map(lambda x: x.rstrip(" (estimated)"))
df["budget"] = df["budget"].map(lambda x: x.replace(",", ""))

In [12]:
# split digit part and character part

df["estimated_budget"] = df["budget"].str.extract("(\d+)")
df["budget_unit"] = df["budget"].str.extract("(\D+)")

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\D'
C:\Users\HIEPNT\AppData\Local\Temp\ipykernel_8784\2208822417.py:3: SyntaxWarning: invalid escape sequence '\d'
  df["estimated_budget"] = df["budget"].str.extract("(\d+)")
C:\Users\HIEPNT\AppData\Local\Temp\ipykernel_8784\2208822417.py:4: SyntaxWarning: invalid escape sequence '\D'
  df["budget_unit"] = df["budget"].str.extract("(\D+)")


In [13]:
# get all the unique values of budget_unit
df["budget_unit"].unique()
# if there exist a \xa0, replace it with a ' '
df["budget_unit"] = df["budget_unit"].map(lambda x: x.replace("\xa0", "").replace("\u202fCFA", ""))
df["budget_unit"].unique()

array(['$', 'nan', 'DEM', '¥', 'NLG', 'RON', '₩', '£', 'AZM', 'CA$', '₱',
       'CHF', 'DKK', 'FIM', 'PTE', 'ROL', 'ESP', 'SEK', '₹', 'PKR', 'EGP',
       'FRF', 'A$', 'TRL', 'XAU', 'NZ$', 'GRD', 'ITL', 'CN¥', 'BEF',
       'IDR', '€', 'NOK', 'MX$', 'LUF', 'IRR', 'RUR', 'HK$', 'THB', 'ZAR',
       'SIT', 'HUF', 'ISK', 'LTL', 'R$', 'DOP', 'MYR', 'LKR', 'HRK',
       'SGD', 'PLN', 'EEK', 'EC$', 'CZK', 'IEP', 'BGL', 'CLP', 'YUM',
       'NT$', 'MMK', '₫', 'VEB', '₪', 'COP', 'SKK', 'BDT', 'MTL', 'ARS',
       'GTQ', 'MVR', 'GEL', 'LVL', 'MAD', 'UAH', 'MOP', 'KZT', 'NPR',
       'NGN', 'JOD', 'PEN', 'BHD', 'PYG', 'ANG', 'BTN', 'MNT', 'AED',
       'TND', 'DZD', 'HNL', 'BAM', 'KPW', 'KES', 'GHC', 'RWF', 'MKD',
       'UZS', 'UGX', 'LBP', 'AMD', 'CDF', 'ALL', 'FCFA', 'KWD'],
      dtype=object)

In [14]:
# Các đơn vị tiền tệ và tỷ lệ chuyển đổi sang đô la
currency_conversion = {
    '$': 1,    # Đô la Mỹ
    'A$': 0.75,  # Đô la Úc
    '£': 1.32,   # Bảng Anh
    'SEK': 0.11,  # Krona Thụy Điển
    'DKK': 0.16,  # Krone Đan Mạch
    '₹': 0.014,  # Rupee Ấn Độ
    '€': 1.18,   # Euro
    'CA$': 0.75,  # Đô la Canada
    'DEM': 0.61,  # Đức Mark
    '¥': 0.007,    # Yên Nhật
    'TRL': 0.13,  # Lira Thổ Nhĩ Kỳ
    'R$': 0.19,   # Real
    'CN¥': 0.15,  # Nhân dân tệ
    'PLN': 0.27,  # Zloty Ba Lan
    'FRF': 0.18,  # Pháp Franc
    '₩': 0.00084, # Won Hàn Quốc
    'FIM': 0.18,  # Markka Phần Lan
    'RUR': 0.013, # Ruble Nga
    'SGD': 0.74,  # Đô la Singapore
    'HUF': 0.0034,# Forint Hungary
    'NOK': 0.11,  # Krone Na Uy
    'MX$': 0.049, # Peso Mexico
    'NZ$': 0.69,  # Đô la New Zealand
    'HK$': 0.13,  # Đô la Hồng Kông
    'IDR': 0.000071, # Rupiah Indonesia
    'ESP': 0.0066, # Peseta Tây Ban Nha
    'ISK': 0.0079, # Króna Iceland
    'CHF': 1.09,  # Franc Thụy Sĩ
    'VEB': 0.0000000000000001, # Bolívar Venezuela
    'ALL': 0.0094, # Lek Albania
    'PKR': 0.0059, # Rupee Pakistan
    'BEF': 0.031, # Franc Bỉ
    'EGP': 0.064, # Bảng Ai Cập
    'DOP': 0.017, # Peso Dominica
    'ZAR': 0.067, # Rand Nam Phi
    'NGN': 0.0026, # Naira Nigeria
    'THB': 0.032, # Baht Thái Lan
    'IRR': 0.000024, # Rial Iran
    'LTL': 0.32, # Litas Litva
    'BDT': 0.012, # Taka Bangladesh
    'MYR': 0.24, # Ringgit Malaysia
    'LKR': 0.0054, # Rupee Sri Lanka
    'NT$': 0.035, # Đô la Đài Loan
    'CZK': 0.044, # Koruna Cộng hòa Séc
    'ARS': 0.011, # Peso Argentina
    'GEL': 0.31, # Lari Georgia
    'MNT': 0.00035, # Tugrik Mông Cổ
    'UZS': 0.000095, # Som Uzbekistan
    '₱': 0.021, # Peso Philipin
    'NLG': 0.59, # Guilder Hà Lan
    'ITL': 0.00061, # Lira Ý
    'NAD': 0.067, # Đô la Namibia
    'ROL': 0.00024, # Leu Rumani
    'EEK': 0.075, # Kroon Estonia
    'GTQ': 0.13, # Quetzal Guatemala
    'HNL': 0.041, # Lempira Honduras
    'XAU': 1.88, # Vàng
    'AED': 0.27, # Dirham UAE
    'COP': 0.00027, # Peso Colombia
    'PEN': 0.29, # Sol Peru
    'MOP': 0.12, # Pataca Ma Cao
    'SIT': 0.0059, # Tolar Slovenia
    'NPR': 0.0085, # Rupee Nepal
    'PTE': 0.0059, # Escudo Bồ Đào Nha
    '₪': 4.06,
    'BGL': 0.61, # Lev Bulgaria
    'MAD': 0.11, # Dirham Ma-rốc
    'UAH': 0.037, # Hryvnia Ukraina
    'MKD': 0.019, # Denar Macedonia
    'KES': 0.0092, # Shilling Kenya
    'HRK': 0.16, # Kuna Croatia
    'KWD': 3.29, # Dinar Kuwait
    'MMK': 0.00074, # Kyat Myanma
    'KZT': 0.0024, # Tenge Kazakhstan
    'AZM': 0.00012, # Manat Azerbaijan
    'GHC': 0.00018, # Cedi Ghana
    'JOD': 1.41, # Dinar Jordan
    '₫': 0.000043, # Đồng Việt Nam
    'ANG': 0.56, # Guilder Antille thuộc Hà Lan
    'LVL': 1.59, # Lats Latvia
    'MVR': 0.064, # Rufiyaa Maldives
    'LUF': 0.031, # Franc Luxembourg
    'UGX': 0.00027, # Shilling Uganda
    'IQD': 0.00084, # Dinar Iraq
    'GRD' : 0.0031, # Drachma Hy Lạp
    'LYD' : 0.23, # Dinar Libya
    'BYR' : 0.000041, # Ruble Belarus
    'AMD' : 0.0021, # Dram Armenia
    'TTD' : 0.15, # Đô la Trinidad và Tobago
    'RON' : 0.24, # Leu Romania
    'ATS' : 0.073, # Schilling Áo
    'BAM' : 0.61, # Mark Bosnia và Herzegovina
    'BSD' : 1, # Đô la Bahamas
    'OMR' : 2.60, # Rial Oman
    'CLP' : 0.0013, # Peso Chile
    'BHD' : 2.65, # Dinar Bahrain
    'SZL' : 0.067, # Lilangeni Swaziland
    'KGS' : 0.012, # Som Kyrgyzstan
    'ZMK' : 0.00011, # Kwacha Zambia
    'SLL' : 0.00010, # Leone Sierra Leone
    'TZS' : 0.00043, # Shilling Tanzania
    'DZD' : 0.0074, # Dinar Algeria
    'F' : 0.18, # Franc Pháp
    'TND' : 0.37, # Dinar Tunisia
    'CDF' : 0.00055, # Franc Congo
    'KHR' : 0.00025, # Riel Campuchia
    'AFA' : 0.014, # Afghani Afghanistan
    'BTN' : 0.013, # Ngultrum Bhutan
    'YUM' : 0.55, # Dinar Nam Tư
    'ETB' : 0.018, # Birr Ethiopia
    'CRC' : 0.0019, # Colón Costa Rica
    'TMM' : 0.29, # Manat Turkmenistan
    'QAR' : 0.27, # Rial Qatar
    'SYP' : 0.00008, # Bảng Syria
    'PYG' : 0.00015, # Guarani Paraguay
    'MUR' : 0.022, # Rupee Mauritius
    'FJD' : 0.45, # Đô la Fiji
    'IEP' : 1.40, # Bảng Ireland
    'SOS' : 0.0017, # Shilling Somalia
    'CUP' : 0.038, # Peso Cuba
    'SDD' : 0.0000166968, # Dinar Sudan
    'ZWD' : 0.0029, # Đô la Zimbabwe
    'EC$' : 0.37, # Đô la Đông Caribê
    'UYU' : 0.023, # Peso Uruguay
    'BOB' : 0.15, # Boliviano Bolivia
    'LBP' : 0.00066, # Bảng Li-băng
    'SKK' : 0.036, # Koruna Slovakia
    'RWF' : 0.0010, # Franc Rwanda
    'BIF' : 0.00052, # Franc Burundi
    'JMD' : 0.0067, # Đô la Jamaica
    'FCFA' : 0.0018, # Franc CFA Trung Phi
    'MTL' : 0.00028, # Lira Malta
    'KPW' : 0.0011, # Won Triều Tiên
    'BBD'  : 0.50, # Đô la Barbados
    'AWG' : 0.56, # Florin Aruba
}

def convert_budget_to_usd(row):
    if row['budget_unit'] in currency_conversion:
        # Convert the extracted budget to float and apply the exchange rate
        return float(row['estimated_budget']) * currency_conversion[row['budget_unit']]
    return float(row['estimated_budget'])

# Apply the conversion function to create a 'budget_usd' column
df["budget_unit"].fillna('USD', inplace=True)  # Assuming missing units are USD
df['budget_usd'] = df.apply(convert_budget_to_usd, axis=1)

In [15]:
#gross processing
df["gross"] = df["gross"].astype(str)
df["gross"] = df["gross"].map(lambda x: x.replace(",", ""))
# split digit part and character part
df["estimated_gross"] = df["gross"].str.extract("(\d+)")
df["gross_unit"] = df["gross"].str.extract("(\D+)")

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\D'
C:\Users\HIEPNT\AppData\Local\Temp\ipykernel_8784\2443639033.py:5: SyntaxWarning: invalid escape sequence '\d'
  df["estimated_gross"] = df["gross"].str.extract("(\d+)")
C:\Users\HIEPNT\AppData\Local\Temp\ipykernel_8784\2443639033.py:6: SyntaxWarning: invalid escape sequence '\D'
  df["gross_unit"] = df["gross"].str.extract("(\D+)")


In [16]:
# get all the unique values of budget_unit
df["gross_unit"].unique()
# if there exist a \xa0, replace it with a ' '
df["gross_unit"] = df["gross_unit"].map(lambda x: x.replace("\xa0", "").replace("\u202fCFA", ""))
df["gross_unit"].unique()

array(['$', 'nan', '€', '₹'], dtype=object)

In [17]:
# Các đơn vị tiền tệ và tỷ lệ chuyển đổi sang đô la
currency_conversion = {
    '$': 1,    # Đô la Mỹ
    '₹': 0.014,  # Rupee Ấn Độ
    '€': 1.18,   # Euro
}

def convert_gross_to_usd(row):
    if row['gross_unit'] in currency_conversion:
        # Convert the extracted budget to float and apply the exchange rate
        return float(row['estimated_gross']) * currency_conversion[row['gross_unit']]
    return float(row['estimated_gross'])

# Apply the conversion function to create a 'budget_usd' column
df["gross_unit"].fillna('USD', inplace=True)  # Assuming missing units are USD
df['gross_usd'] = df.apply(convert_gross_to_usd, axis=1)

In [18]:
# Extract wins and nominations and create new columns
wins = df['win_and_nomination'].str.extract(r'(\d+) wins?')
nominations = df['win_and_nomination'].str.extract(r'(\d+) nominations?')

# Fill missing values with 0
wins.fillna(0, inplace=True)
nominations.fillna(0, inplace=True)

# Add the extracted columns to the DataFrame
df['win'] = wins
df['nomination'] = nominations

In [19]:
# url -> movie_id
df['movie_id'] = df['url'].str.split('/').str[-2]

In [20]:
# drop column budget_unit, gross_unit, win_and_nomination
df = df.drop(['budget_unit', 'gross_unit', 'win_and_nomination'], axis=1)

In [21]:
# delete duplicate movies by movie_id
df = df.drop_duplicates(subset=['movie_id'])
# check if there is any duplicate movie_id
df['movie_id'].duplicated().sum()

0

In [22]:
df.drop(['url', 'budget', 'gross', 'estimated_budget','estimated_gross'], axis=1, inplace=True)

In [23]:
# export data frame to csv and save in cleaned_data folder
df.to_csv('../cleaned_data/cleaned_movie_data.csv', index=False)